# Trade Demo

**Summary:** In this demo, a data scientist wants to be able to determine that the amount of goods exported from a handful of nations (usa, canada, netherlands, united kingdom, and italy) matches the amount of goods those nations claim to have imported from each other. We want to return a list of commodities where the ratio of expected imports / exports is off by 10% or more. Importantly, the data scientist should be able to do this:

- without requiring any nation to disclose to anyone the amount of any particular good they have imported or exported (unless they're in violation)
- without needing a data compliance officer to manually accept any .get() requests.

In [128]:
import pandas as pd
schema = pd.read_csv('datasets/schema.csv')

canada = pd.read_csv('datasets/ca - feb 2021.csv')
italy =  pd.read_csv('datasets/it - feb 2021.csv')
netherlands =  pd.read_csv('datasets/nl - feb 2021.csv')
united_states = pd.read_csv('datasets/us - feb 2021.csv')

/Users/andrewliamtrask/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Step 1: Load The Dataset

We have trade data from 4 countries, all of which have provided data from Feb 2021. They key colums are:

- Commodity Code: the official code of that type of good
- Reporter: the country claiming the import/export value
- Partner: the country being claimed about
- Trade Flow: the direction of the goods being reported about (imports, exports, etc)
- Trade Value (US$): the declared USD value of the good

So if we considered the following row, it specifies that Canada thinks that it imports $1,955,175 USD of "Cocoa and cocoa preparations" from the United Kingdom.

In [130]:
canada[2:3]

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0


### Step 2: Spin Up Domain Nodes

As the main requirement of this demo is that none of these countries should be sharing their data with anyone else, each country will get their own domain node to hold/protect their data while it's under study. So, we need to spin up 4 domain nodes.

Navigate to PySyft/grid and run the following commands in separate terminals:

```bash
docker-compose up --type "Domain" --port 5000 --name "Canada"
```

```bash
docker-compose up --type "Domain" --port 5001 --name "United States"
```

```bash
docker-compose up --type "Domain" --port 5002 --name "Italy"
```

```bash
docker-compose up --type "Domain" --port 5003 --name "Netherlands"
```

Additionally we'll need to setup a Network node which will help us interact with the remote data

```bash
docker-compose up --type "Network" --port 5004 --name "United Nations Global Platform"
```

### Step 3: Register domain nodes with the United Nations network!

You may if you wish reset the default ADMIN username/password using the user interface. For now, we'll just leave it as is to keep things easy and login using the defaults set by PyGrid (info@openmined.org and changethis).

In [ ]:
import syft as sy

# Login: (defaults to localhost if you don't specify a url)
ca = sy.login(username="info@openmined.org", password="changethis", port=5000)
usa = sy.login(username="info@openmined.org", password="changethis", port=5001)
it = sy.login(username="info@openmined.org", password="changethis", port=5002)
ne = sy.login(username="info@openmined.org", password="changethis", port=5003)

un = sy.login(username="info@openmined.org", password="changethis", port=5004)

```
Connecting to Canada... connected!         ...logging in as info@openmined.org... logged in!
Connecting to United States... connected!  ...logging in as info@openmined.org... logged in!
Connecting to Italy... connected!          ...logging in as info@openmined.org... logged in!
Connecting to Netherlands... connected!    ...logging in as info@openmined.org... logged in!
Connecting to United Nations... connected! ...logging in as info@openmined.org... logged in!
```

In [135]:
# Each domain admin applies to join the Network (so that users can find them!)
ca.apply_to_network(url="localhost", port=5004, reason="This is Sue Grafton. Per our phone convo, we'd like to join your network.", contact="sue@canada.ca")
usa.apply_to_network(url="localhost", port=5004, reason="We were connected before, just need to re-establish with new node", contact="john@usa.gov"))
it.apply_to_network(url="localhost", port=5004, reason="We heard great things about the UNGP and would like to participate", contact="suzy@it.it"))
ne.apply_to_network(url="localhost", port=5004, reason="We recently spun up a domain and would like to join the network.", contact="bill@ne.ne"))

```
Application submitted from Canada -> United Nations!
You'll get an email (sue@canada.ca) when your application has been processed!

Application submitted from United States -> United Nations!
You'll get an email (john@usa.gov) when your application has been processed!

Application submitted from Italy -> United Nations!
You'll get an email (suzy@it.it) when your application has been processed!

Application submitted from Netherlands -> United Nations!
You'll get an email (bill@ne.ne) when your application has been processed!
```

In [ ]:
# United Nations admin checks network affiliation applications upon receiving 4 emails that 4 applications have been received!


/Users/andrewliamtrask/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [120]:
canada[0:5]

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


In [30]:
# Non matching example
italy[italy['Commodity Code'] == "18"][italy['Partner'] == "Canada"]
canada[canada['Commodity Code'] == "18"][canada['Partner'] == "Italy"]

<ipython-input-30-0551d926d722>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  italy[italy['Commodity Code'] == "18"][italy['Partner'] == "Canada"]
<ipython-input-30-0551d926d722>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  canada[canada['Commodity Code'] == "18"][canada['Partner'] == "Italy"]


,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
32291,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Italy,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,NaN,1010792,0
55735,HS,2021,202102,February 2021,2,0,2,Exports,124,Canada,...,Italy,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2063200,0


In [85]:
canada[canada['Commodity Code'] == str(551090)][canada['Partner'] == "United States"]

<ipython-input-85-893c56eeae87>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  canada[canada['Commodity Code'] == str(551090)][canada['Partner'] == "United States"]


,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag


In [99]:
canada_commodities = set(canada[canada['Partner'] == 'United States of America']['Commodity Code'])
usa_commoddities = set(united_states[united_states['Partner'] == 'Canada']['Commodity Code'])

In [111]:
for i in canada_commodities.intersection(usa_commoddities):
    us_imports = united_states[united_states['Commodity Code'] == str(i)][united_states['Partner'] == "Canada"][united_states['Trade Flow'] == "Imports"]
    
    canada_exports = canada[canada['Commodity Code'] == str(i)][canada['Partner'] == 'United States of America'][canada['Trade Flow'] == "Exports"]
    
    if(len(us_imports) > 0 and len(canada_exports) > 0):
        us_thinks_it_imports_from_canada = int(us_imports['Trade Value (US$)'])
        canada_thinks_it_exports_to_us = int(canada_exports['Trade Value (US$)'])        
        print(i, us_thinks_it_imports_from_canada, canada_thinks_it_exports_to_us, "\t" + str(us_imports['Commodity']).replace("\n"," ").split("  ")[2].split("Name")[0])
        
        
        


<ipython-input-111-6b1c5780330f>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  us_imports = united_states[united_states['Commodity Code'] == str(i)][united_states['Partner'] == "Canada"][united_states['Trade Flow'] == "Imports"]
<ipython-input-111-6b1c5780330f>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  canada_exports = canada[canada['Commodity Code'] == str(i)][canada['Partner'] == 'United States of America'][canada['Trade Flow'] == "Exports"]


84 1660673548 1863126155 	Nuclear reactors, boilers, machinery and mecha... 
41 1046772 1830656 	Raw hides and skins (other than furskins) and ... 
26 65241464 63228517 	Ores, slag and ash 
8540 366408 393892 	Thermionic, cold cathode or photo-cathode valv... 
04 16463987 16482274 	Dairy produce; birds' eggs; natural honey; edi... 
52 1276643 2117212 	Cotton 
06 36414025 37175452 	Trees and other plants, live; bulbs, roots and... 
35 17486000 18238532 	Albuminoidal substances; modified starches; gl... 
84 1660673548 1863126155 	Nuclear reactors, boilers, machinery and mecha... 
50 6389 14027 	Silk 
051110 1787483 1782096 	Animal products; bovine semen 
2918 11562838 1139550 	Acids; carboxylic acid with additional oxygen ... 
53 58049 202663 	Vegetable textile fibres; paper yarn and woven... 
46 192282 3688299 	Manufactures of straw, esparto or other plaiti... 
33 111101126 125346608 	Essential oils and resinoids; perfumery, cosme... 
32 78485382 84396873 	Tanning or dyeing extracts; ta

KeyboardInterrupt: 

In [ ]:
# canada[canada['Commodity Code'] == "18"][canada['Partner'] == "United States"]